#  Question 1 (20%)
Load the energy data from the file Energy Indicators.xls, which is a list of indicators of energy supply and renewable electricity production from the United Nations for the year 2013, and should be put into a DataFrame with the variable name of energy.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

Convert Energy Supply to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as np.NaN values.

Rename the following list of countries (for use in later questions):

"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these,

e.g.

'Bolivia (Plurinational State of)' should be 'Bolivia',

'Switzerland17' should be 'Switzerland'.



Next, load the GDP data from the file world_bank.csv, which is a csv containing countries' GDP from 1960 to 2015 from World Bank. Call this DataFrame GDP.

Make sure to skip the header, and rename the following list of countries:

"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"



Finally, load the Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology from the file scimagojr-3.xlsx, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame ScimEn.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15).

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

This function should return a DataFrame with 20 columns and 15 entries.

In [2]:
import pandas as pd
import numpy as np
pd.set_option("max.rows", 500)
pd.set_option("max.columns", 500)


energy = pd.read_excel(
    r"Energy Indicators.xls", skiprows=17, skip_footer=38, parse_cols="C:F"
)
country_list = ["Country", "Energy Supply", "Energy Supply per Capita", "% Renewable"]
energy.columns = country_list
energy["Energy Supply"] = energy["Energy Supply"] * 10 ** 7
# energy.replace( '...',np.nan, inplace=True)
# energy.loc[energy['Energy Supply'] == '...'] = np.NaN
energy[
    ["Country", "Energy Supply", "Energy Supply per Capita", "% Renewable"]
] = energy[
    ["Country", "Energy Supply", "Energy Supply per Capita", "% Renewable"]
].replace(
    to_replace="...", value=np.nan
)
# energy[['Energy Supply', 'Energy Supply per Capita']] = energy[['Energy Supply', 'Energy Supply per Capita']].apply(pd.to_numeric)
# energy['Country'] = energy['Country'].replace({"Republic of Korea": "South Korea", "United States of America": "United States", "United Kingdom of Great Britain and Northern Ireland": "United Kingdom", "China, Hong Kong Special Administrative Region": "Hong Kong"})
# energy['Country']
energy["Country"] = energy["Country"].str.replace(r" \(.*\)", "")
energy["Country"] = energy["Country"].str.replace(r"([0-9]+)$", "")
new_name = {
    "Republic of Korea": "South Korea",
    "United States of America": "United States",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "China, Hong Kong Special Administrative Region": "Hong Kong",
}
energy["Country"].replace(to_replace=new_name, inplace=True)
energy.reset_index()
energy = energy.set_index("Country")


GDP = pd.read_csv(r"API_NY.GDP.MKTP.CD_DS2_en_csv_v2_1120928.csv", skiprows=4)
new_country_name = {
    "Korea, Rep.": "South Korea",
    "Iran, Islamic Rep.": "Iran",
    "Hong Kong SAR, China": "Hong Kong",
}
GDP["Country Name"].replace(to_replace=new_country_name, inplace=True)

years_needed = np.arange(2006, 2016).astype(str)
GDP = GDP[np.append(["Country Name"], years_needed)]
GDP.reset_index()

GDP = GDP.rename(columns={"Country Name": "Country"})
GDP = GDP.set_index("Country")

ScimEn = pd.read_excel(r"scimagojr country rank 1996-2019.xlsx", header=0)
ScimEn.reset_index()
ScimEn = ScimEn.set_index("Country")
ScimEn = ScimEn[:16]
first_merge = pd.merge(energy, GDP, how="inner", on="Country")
result = pd.merge(first_merge, ScimEn, how="inner", on="Country")
result.head(5)


# first_merge = pd.merge(energy, GDP, how='inner', left_index=True, right_index=True)
# result = pd.merge(ScimEn, first_merge, how='inner', left_index=True, right_index=True)
# result = result.reset_index().dropna(thresh=result.shape[1]-10).set_index('Country')
# result = result.loc[result['Rank']<=15]
# result.shape

C:\Users\plams\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: the 'parse_cols' keyword is deprecated, use 'usecols' instead
  
C:\Users\plams\Anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: the 'skip_footer' keyword is deprecated, use 'skipfooter' instead
  return func(*args, **kwargs)


,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,Rank,Region,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
Country,,,,,,,,,,,,,,,,,,,,,
Australia,53860000000,231.0,11.81081,7.460542e+11,8.530996e+11,1.053996e+12,9.278052e+11,1.146138e+12,1.396650e+12,1.546152e+12,1.576184e+12,1.467484e+12,1.351694e+12,14,Pacific Region,18077,17661,263733,44799,14.59,161
Brazil,121490000000,59.0,69.64803,1.107640e+12,1.397084e+12,1.695825e+12,1.667020e+12,2.208872e+12,2.616201e+12,2.465189e+12,2.472807e+12,2.455994e+12,1.802214e+12,15,Latin America,18024,17745,152380,37112,8.45,121
Canada,104310000000,296.0,61.94543,1.315415e+12,1.464977e+12,1.549131e+12,1.371153e+12,1.613543e+12,1.789141e+12,1.823967e+12,1.842018e+12,1.801480e+12,1.552900e+12,8,Northern America,29633,29011,491467,88987,16.59,210
China,1271910000000,93.0,19.75491,2.752132e+12,3.550342e+12,4.594307e+12,5.101702e+12,6.087165e+12,7.551500e+12,8.532231e+12,9.570406e+12,1.043853e+13,1.101554e+13,1,Asiatic Region,235126,233883,1909601,1306438,8.12,224
France,105970000000,166.0,17.02028,2.318594e+12,2.657213e+12,2.918383e+12,2.690222e+12,2.642610e+12,2.861408e+12,2.683825e+12,2.811078e+12,2.852166e+12,2.438208e+12,11,Western Europe,22429,21913,300015,58151,13.38,169


In [2]:
def answer_one():
    import pandas as pd
    import numpy as np

    pd.set_option("max.rows", 500)
    pd.set_option("max.columns", 500)


    energy = pd.read_excel('Energy Indicators.xls', skip_footer=38, skiprows=17, parse_cols='C:F')
    col_names = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy.columns = col_names
    energy.loc[energy['Energy Supply'] == '...'] = np.NaN
    energy[['Energy Supply', 'Energy Supply per Capita']] = energy[['Energy Supply', 'Energy Supply per Capita']].apply(pd.to_numeric)
    energy['Energy Supply'] = energy['Energy Supply']*10**6
    energy['Country'] = energy['Country'].str.replace(r" \(.*\)","")
    energy['Country'] = energy['Country'].str.replace(r"([0-9]+)$","")
    replace_dict={"Republic of Korea": "South Korea",
                  "United States of America": "United States",
                  "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
                  "China, Hong Kong Special Administrative Region": "Hong Kong"}
    energy['Country'].replace(to_replace=replace_dict, inplace=True)
    energy.reset_index()
    energy = energy.set_index('Country')
    
    GDP = pd.read_csv('API_NY.GDP.MKTP.CD_DS2_en_csv_v2_1120928.csv', skiprows=4)
    replace_dict = {"Korea, Rep.": "South Korea", 
                    "Iran, Islamic Rep.": "Iran",
                    "Hong Kong SAR, China": "Hong Kong"
                   }
    GDP['Country Name'].replace(to_replace=replace_dict, inplace=True)
    years_to_keep = np.arange(2006, 2016).astype(str)
    GDP = GDP[np.append(['Country Name'],years_to_keep)]
    GDP.reset_index()
    GDP = GDP.rename(columns={'Country Name': 'Country'})
    GDP = GDP.set_index('Country')
    
    ScimEn = pd.read_excel('scimagojr country rank 1996-2019.xlsx', header=0)
    ScimEn.reset_index()
    ScimEn = ScimEn.set_index('Country')
    first_merge = pd.merge(energy, GDP, how='outer', left_index=True, right_index=True)
    result = pd.merge(ScimEn, first_merge, how='outer', left_index=True, right_index=True)
    result = result.reset_index().dropna(thresh=result.shape[1]-10).set_index('Country')
    result = result.loc[result['Rank']<=15]
    
    return result
answer_one()


C:\Users\plams\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: the 'parse_cols' keyword is deprecated, use 'usecols' instead
  if __name__ == '__main__':


,Rank,Region,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,,
Australia,14.0,Pacific Region,18077.0,17661.0,263733.0,44799.0,14.59,161.0,5.386000e+09,231.0,11.810810,7.460542e+11,8.530996e+11,1.053996e+12,9.278052e+11,1.146138e+12,1.396650e+12,1.546152e+12,1.576184e+12,1.467484e+12,1.351694e+12
Brazil,15.0,Latin America,18024.0,17745.0,152380.0,37112.0,8.45,121.0,1.214900e+10,59.0,69.648030,1.107640e+12,1.397084e+12,1.695825e+12,1.667020e+12,2.208872e+12,2.616201e+12,2.465189e+12,2.472807e+12,2.455994e+12,1.802214e+12
Canada,8.0,Northern America,29633.0,29011.0,491467.0,88987.0,16.59,210.0,1.043100e+10,296.0,61.945430,1.315415e+12,1.464977e+12,1.549131e+12,1.371153e+12,1.613543e+12,1.789141e+12,1.823967e+12,1.842018e+12,1.801480e+12,1.552900e+12
China,1.0,Asiatic Region,235126.0,233883.0,1909601.0,1306438.0,8.12,224.0,1.271910e+11,93.0,19.754910,2.752132e+12,3.550342e+12,4.594307e+12,5.101702e+12,6.087165e+12,7.551500e+12,8.532231e+12,9.570406e+12,1.043853e+13,1.101554e+13
France,11.0,Western Europe,22429.0,21913.0,300015.0,58151.0,13.38,169.0,1.059700e+10,166.0,17.020280,2.318594e+12,2.657213e+12,2.918383e+12,2.690222e+12,2.642610e+12,2.861408e+12,2.683825e+12,2.811078e+12,2.852166e+12,2.438208e+12
Germany,6.0,Western Europe,32935.0,32227.0,367356.0,78265.0,11.15,186.0,1.326100e+10,165.0,17.901530,2.992197e+12,3.421229e+12,3.730028e+12,3.397791e+12,3.396354e+12,3.744409e+12,3.527345e+12,3.732743e+12,3.883920e+12,3.360550e+12
India,4.0,Asiatic Region,39893.0,38848.0,368175.0,123446.0,9.23,171.0,3.319500e+10,26.0,14.969080,9.402599e+11,1.216735e+12,1.198896e+12,1.341887e+12,1.675615e+12,1.823050e+12,1.827638e+12,1.856722e+12,2.039127e+12,2.103588e+12
Iran,12.0,Middle East,19371.0,19085.0,242250.0,77012.0,12.51,127.0,9.172000e+09,119.0,5.707721,2.662989e+11,3.498816e+11,4.060709e+11,4.140591e+11,4.870696e+11,5.835004e+11,5.988534e+11,4.674149e+11,4.344746e+11,3.858745e+11
Italy,9.0,Western Europe,23725.0,22819.0,312631.0,76446.0,13.18,155.0,6.530000e+09,109.0,33.667230,1.947920e+12,2.210293e+12,2.398857e+12,2.191242e+12,2.134018e+12,2.291991e+12,2.087077e+12,2.141315e+12,2.159134e+12,1.835899e+12


# Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

This function should return a single number.

In [5]:
no_col_reduction = pd.merge(pd.merge(energy, GDP, how="outer", on="Country"), ScimEn, on="Country")
col_with_reduction = pd.merge(pd.merge(energy, GDP, on="Country"), ScimEn, on="Country")
len(no_col_reduction) - len(col_with_reduction)

0

# Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

This function should return a Series named avgGDP with 15 countries and their average GDP sorted in descending order.

In [16]:
import pandas as pd
def answer_three():
    Top15 = answer_one()
    avgGDP = Top15.iloc[:,11:].mean(axis=1).sort_values(ascending=False)
    return pd.Series(avgGDP)
    
answer_three()

C:\Users\plams\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: the 'parse_cols' keyword is deprecated, use 'usecols' instead
  if __name__ == '__main__':


Country
United States         1.566858e+13
China                 6.919386e+12
Japan                 5.177131e+12
Germany               3.518657e+12
United Kingdom        2.776607e+12
France                2.687371e+12
Italy                 2.139775e+12
Brazil                1.988885e+12
Russian Federation    1.668067e+12
Canada                1.612373e+12
India                 1.602352e+12
Spain                 1.398587e+12
Australia             1.206526e+12
South Korea           1.165810e+12
Iran                  4.393498e+11
dtype: float64

# Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

This function should return a single number.

In [ ]:
def answer_four():
    Top15 = answer_one()
    Top15["avgGDP"] = answer_three()
    Top15.sort_values("avgGDP", ascending=False,inplace=True)
    gdp_ini = Top15.iloc[5]['2006']
    gdp_fin = Top15.iloc[5]['2015']
    del_change = (gdp_fin - gdp_ini)
    return abs(del_change)
answer_four()
    

# Question 5 (6.6%)
What is the mean Energy Supply per Capita?

This function should return a single number.

In [30]:
def answer_five():
    Top15 = answer_one()
    val = Top15["Energy Supply per Capita"].mean()
    return val
answer_five()

C:\Users\plams\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: the 'parse_cols' keyword is deprecated, use 'usecols' instead
  if __name__ == '__main__':


157.6

# Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

This function should return a tuple with the name of the country and the percentage.

In [33]:
def answer_six():
    Top15 = answer_one()
    return Top15['% Renewable'].argmax(), Top15['% Renewable'].max()
answer_six()

# Question 7 (6.6%)¶
Create a new column that is the ratio of Self-Citations to Total Citations. What is the maximum value for this new column, and what country has the highest ratio?

This function should return a tuple with the name of the country and the ratio.

In [35]:
def answer_seven():
    Top15 = answer_one()
    Top15['cit_ratio'] = Top15['Self-citations']/Top15['Citations']
    return Top15['cit_ratio'].argmax(), Top15['cit_ratio'].max()
answer_seven()

C:\Users\plams\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: the 'parse_cols' keyword is deprecated, use 'usecols' instead
  if __name__ == '__main__':
C:\Users\plams\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  after removing the cwd from sys.path.


('China', 0.6841418704745127)

# Question 8 (6.6%)
Create a column that estimates the population using Energy Supply and Energy Supply per capita. What is the third most populous country according to this estimate?

This function should return a single string value.

In [39]:
def answer_eight():
    Top15 = answer_one()
    Top15['pop_estimate'] = Top15['Energy Supply']/Top15['Energy Supply per Capita']
    country = Top15.sort_values('pop_estimate', ascending=False, inplace=True)
    Top15 = Top15.reset_index()
    return Top15.loc[2, 'Country']
answer_eight()

C:\Users\plams\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: the 'parse_cols' keyword is deprecated, use 'usecols' instead
  if __name__ == '__main__':
C:\Users\plams\Anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: the 'skip_footer' keyword is deprecated, use 'skipfooter' instead
  return func(*args, **kwargs)


'United States'

# Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the .corr() method, (Pearson's correlation).

This function should return a single number.

(Optional: Use the built-in function plot9() to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)

In [47]:
def answer_nine():
    Top15 = answer_one()
    Top15['pop_estimate'] = Top15['Energy Supply']/Top15['Energy Supply per Capita']
    Top15['cit_doc_per_capita'] = Top15['Citable documents']/Top15['pop_estimate']
    return Top15['cit_doc_per_capita'].corr(Top15['Energy Supply per Capita'])
answer_nine()


C:\Users\plams\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: the 'parse_cols' keyword is deprecated, use 'usecols' instead
  if __name__ == '__main__':


0.7467513245295704

# Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

This function should return a series named HighRenew whose index is the country name sorted in ascending order of rank.

In [62]:
def answer_ten():
    Top15 = answer_one()
    threshold = Top15['% Renewable'].median()
    Top15['HighRenew'] = Top15['% Renewable'].apply(lambda x: 1 if x > threshold else 0)
    Top15.sort_values("Rank", ascending=True, inplace=True)
    return Top15['HighRenew']
answer_ten()


C:\Users\plams\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: the 'parse_cols' keyword is deprecated, use 'usecols' instead
  if __name__ == '__main__':
C:\Users\plams\Anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: the 'skip_footer' keyword is deprecated, use 'skipfooter' instead
  return func(*args, **kwargs)


Country
China                 1
United States         0
Japan                 0
India                 0
United Kingdom        0
Germany               1
Russian Federation    1
Canada                1
Italy                 1
South Korea           0
France                0
Iran                  0
Spain                 1
Australia             0
Brazil                1
Name: HighRenew, dtype: int64

# Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
This function should return a DataFrame with index named Continent ['Asia', 'Australia', 'Europe', 'North America', 'South America'] and columns ['size', 'sum', 'mean', 'std']

In [71]:
def answer_eleven():
    import numpy as np
    import pandas as pd
    Top15 = answer_one()
    ContinentDict  = {'China':'Asia',
                      'United States':'North America', 
                      'Japan':'Asia',
                      'United Kingdom':'Europe',
                      'Russian Federation':'Europe',
                      'Canada':'North America',
                      'Germany':'Europe', 
                      'India':'Asia',
                      'France':'Europe', 
                      'South Korea':'Asia', 
                      'Italy':'Europe', 
                      'Spain':'Europe', 
                      'Iran':'Asia',
                      'Australia':'Australia', 
                      'Brazil':'South America'}
    Top15 = Top15.reset_index()
    Top15['Continent'] = Top15['Country'].map(ContinentDict)
    Top15['Pop_est'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    result = Top15.copy()
    result = result[['Continent', 'Pop_est']]
    result = result.groupby('Continent')['Pop_est'].agg({'size': np.size,'sum': np.sum,'mean': np.mean,'std': np.std})
    #result = grouped.agg(['np.size', 'sum', 'mean', 'std'])
    idx = pd.IndexSlice
    #result = result.loc[:, idx['PopEst']]
    #result = result.reset_index()
    #result = result.set_index('Continent')
    return result

answer_eleven()
    

C:\Users\plams\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: the 'parse_cols' keyword is deprecated, use 'usecols' instead
  if __name__ == '__main__':
C:\Users\plams\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


,size,sum,mean,std
Continent,,,,
Asia,5.0,2.898666e+09,5.797333e+08,6.790979e+08
Australia,1.0,2.331602e+07,2.331602e+07,NaN
Europe,6.0,4.579297e+08,7.632161e+07,3.464767e+07
North America,2.0,3.528552e+08,1.764276e+08,1.996696e+08
South America,1.0,2.059153e+08,2.059153e+08,NaN


# Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

This function should return a Series with a MultiIndex of Continent, then the bins for % Renewable. Do not include groups with no countries.

In [72]:
def answer_twelve():
    Top15 = answer_one()
    ContinentDict  = {'China':'Asia',
                      'United States':'North America', 
                      'Japan':'Asia',
                      'United Kingdom':'Europe',
                      'Russian Federation':'Europe',
                      'Canada':'North America',
                      'Germany':'Europe', 
                      'India':'Asia',
                      'France':'Europe', 
                      'South Korea':'Asia', 
                      'Italy':'Europe', 
                      'Spain':'Europe', 
                      'Iran':'Asia',
                      'Australia':'Australia', 
                      'Brazil':'South America'}
    
    Top15["bins"] = pd.cut(Top15["% Renewable"], 5)
    return Top15.groupby([ContinentDict, Top15['bins']]).size()
answer_twelve()
    
    

C:\Users\plams\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: the 'parse_cols' keyword is deprecated, use 'usecols' instead
  if __name__ == '__main__':
C:\Users\plams\Anaconda3\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: the 'skip_footer' keyword is deprecated, use 'skipfooter' instead
  return func(*args, **kwargs)


               bins            
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
Australia      (2.212, 15.753]     1
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
North America  (2.212, 15.753]     1
               (56.174, 69.648]    1
South America  (56.174, 69.648]    1
dtype: int64

# Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

This function should return a Series PopEst whose index is the country name and whose values are the population estimate string.

In [76]:
def answer_thirteen():
    Top15 = answer_one()
    Top15["pop_est"] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['pop_est'] =  Top15['pop_est'].map('{:,.8f}'.format)
    return Top15['pop_est']
answer_thirteen()
    
    

C:\Users\plams\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: the 'parse_cols' keyword is deprecated, use 'usecols' instead
  if __name__ == '__main__':


Country
Australia                23,316,017.31601731
Brazil                  205,915,254.23728815
Canada                   35,239,864.86486486
China                 1,367,645,161.29032254
France                   63,837,349.39759036
Germany                  80,369,696.96969697
India                 1,276,730,769.23076916
Iran                     77,075,630.25210084
Italy                    59,908,256.88073394
Japan                   127,409,395.97315437
Russian Federation      143,500,000.00000000
South Korea              49,805,429.86425339
Spain                    46,443,396.22641510
United Kingdom           63,870,967.74193548
United States           317,615,384.61538464
Name: pop_est, dtype: object

# Alternative Q:13

In [74]:
def answer_thirteen():
    Top15 = answer_one()
    Top15["Population"] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    #Top15["Population"] = Top15["Population"]
    
    return Top15['Population'].apply(lambda x: '{0:,}'.format(x))
answer_thirteen()

C:\Users\plams\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: the 'parse_cols' keyword is deprecated, use 'usecols' instead
  if __name__ == '__main__':


Country
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Canada                  35,239,864.86486486
China                 1,367,645,161.2903225
France                  63,837,349.39759036
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
Iran                    77,075,630.25210084
Italy                  59,908,256.880733944
Japan                  127,409,395.97315437
Russian Federation            143,500,000.0
South Korea            49,805,429.864253394
Spain                    46,443,396.2264151
United Kingdom         63,870,967.741935484
United States          317,615,384.61538464
Name: Population, dtype: object